In [405]:

import numpy as np 
import sympy as sp 
from sympy.abc import a,b,c


In [406]:
from functions2 import *

In [407]:

t, x, y, z, c = sp.symbols("t x y z c")
u1, u2, w1, w2 = sp.symbols("u1, u2, w1, w2")
τ, ξ, γ, ζ = sp.symbols("τ, ξ, γ, ζ")
φ, χ, ψ, ω = sp.symbols("φ, χ, ψ, ω")
lvar = [t, x, y, z]
lwlf = sp.Matrix([u1, u2, w1, w2])
var = sp.Matrix([1, t, x, y, z])

q, p = sp.symbols("q p")

β1 = sp.Function("β_1")(t, x, y, z)
β2 = sp.Function("β_2")(t, x, y, z)
β3 = sp.Function("β_3")(t, x, y, z)
β4 = sp.Function("β_4")(t, x, y, z) 


In [408]:
#Constants in case needed
k = [   sp.symbols(f"{symbol}_{index}")
        for symbol in ("k")
        for index in range(0, 15)
        ]
#Constants used for φ,χ,ψ,ω
f = [   sp.symbols(f"{symbol}_{index}")
        for symbol in ("f")
        for index in range(0, 15)
        ]

#Symbols used to represent the partial derivatives of the vector fields
der = [
        sp.symbols(f"{symbol}_{index}")
        for symbol in ("δ")
        for index in ("t", "x", "y", "z", "u1", "u2", "w1", "w2")
        ]

In [409]:
def clear_zer_tn(exp):
    s = np.size(exp)
    l = []
    for i in range(0, s):
        if exp[i] != 0:
            l.append(exp[i])
    #l = np.array(l)
    return l

In [410]:
def listify(expr):
    l = []
    for i in expr:
        l.append(i)
    return l

In [411]:
def comb(τ, ξ, γ, ζ):
    return sp.Matrix([τ, ξ, γ, ζ])
def uncomb(eqs):
    return eqs[0], eqs[1], eqs[2], eqs[3]

In [412]:
grid = var*var.T
grid

Matrix([
[1,    t,    x,    y,    z],
[t, t**2,  t*x,  t*y,  t*z],
[x,  t*x, x**2,  x*y,  x*z],
[y,  t*y,  x*y, y**2,  y*z],
[z,  t*z,  x*z,  y*z, z**2]])

In [413]:
gvar = sp.Matrix( clear_zer_tn ( rm_red_t(  flatten(grid))))
gvar.reshape(3, 5)

Matrix([
[   1,   t,    x,   y,    z],
[t**2, t*x,  t*y, t*z, x**2],
[ x*y, x*z, y**2, y*z, z**2]])

In [414]:
lc0 = [
        sp.symbols(f"{symbol}_{index}")
        for symbol in ("a")
        for index in range(0, 15)
        ]
lc1 = [
        sp.symbols(f"{symbol}_{index}")
        for symbol in ("b")
        for index in range(0, 15)
        ]
lc2 = [
        sp.symbols(f"{symbol}_{index}")
        for symbol in ("c")
        for index in range(0, 15)
        ]
lc3 = [
        sp.symbols(f"{symbol}_{index}")
        for symbol in ("d")
        for index in range(0, 15)
        ]
lc = lc0+lc1+lc2+lc3

In [415]:
#Generate the Ansatz of 2nd order polynomial for our functions
τ = sp.Matrix(lc0).T*gvar
ξ = sp.Matrix(lc1).T*gvar
γ = sp.Matrix(lc2).T*gvar
ζ = sp.Matrix(lc3).T*gvar

In [416]:
#Set certain combations to 0 we have found not to appear
dicti = {
    lc1[13]:0,
    lc2[11]:0,
    lc3[10]:0,
}

In [417]:
eqs = comb(τ, ξ, γ, ζ).subs(dicti)
eqs

Matrix([
[a_0 + a_1*t + a_10*x*y + a_11*x*z + a_12*y**2 + a_13*y*z + a_14*z**2 + a_2*x + a_3*y + a_4*z + a_5*t**2 + a_6*t*x + a_7*t*y + a_8*t*z + a_9*x**2],
[           b_0 + b_1*t + b_10*x*y + b_11*x*z + b_12*y**2 + b_14*z**2 + b_2*x + b_3*y + b_4*z + b_5*t**2 + b_6*t*x + b_7*t*y + b_8*t*z + b_9*x**2],
[           c_0 + c_1*t + c_10*x*y + c_12*y**2 + c_13*y*z + c_14*z**2 + c_2*x + c_3*y + c_4*z + c_5*t**2 + c_6*t*x + c_7*t*y + c_8*t*z + c_9*x**2],
[           d_0 + d_1*t + d_11*x*z + d_12*y**2 + d_13*y*z + d_14*z**2 + d_2*x + d_3*y + d_4*z + d_5*t**2 + d_6*t*x + d_7*t*y + d_8*t*z + d_9*x**2]])

In [418]:
#This would be the sorting/mapping function
#deprecated apparently
def sort_poly(expr, sym):
    #sym = lvar
    #obtain the constant part of the expression
    const = expr.copy()
    for i in sym:
        const = const.subs( i , 0 )

    lista = []
    lista.append(const)
    #substract the constant part every time
    for i in sym:
        expr_zer = expr.copy() - const
        for j in sym:
            #substitute all coefficient with 0
            #except the one we are interested in
            if i != j:
                expr_zer = expr_zer.subs( j, 0 )
        #replace the coefficient we are interested in with 1
        #to obtain a clean final expression
        expr_zer = expr_zer.subs( i , 1 )
        lista.append(expr_zer)
    return lista


In [419]:
#Make a dictionary out of paired coefficients or alone standing ones. Works in case of 0,1,2 coefficients together. 
#In this specific case scenario, it's the only amount we expect to encounter anyways
def mkdic(expr, lcons):
    dic = {}
    c = []
    for i in lcons:
        if expr.has(i):
            c.append(i)
    if len(c) == 1:
        dic.update({c[0]:0})
    elif len(c) != 0: 
        #make it for ANY length?
        #print size, to check if its necessary. for more coefficients its easy to implement actually.
        #print(np.size(c))
        poly = sp.Poly(expr, c[0], c[1])
        coef = poly.coeffs()
        dic.update({ c[1] : -1 * ( coef[0]/coef[1] ) * c[0] })
    else:
        print("No constant found")
    return dic

In [420]:
#Create the dictionary for an expression 
def comp( expr, lvar, lc, dicti):
    #p = sort_poly(p, lvar)
    #i use the variables t,x,y,z as generators for the polynomial
    p = sp.Poly(expr, *lvar)
    p = p.coeffs()
    #p now is a list with expressions in each entry with 0,1 or 2 symbols added/subtracted. 
    #Thats why mkdic processes each entry of this list separately
    s = np.size(p)
    dic = {}
    for i in range(0, s):
        dic.update( mkdic( p[i], lc) )
    dicti.update(dic)
    return dicti

In [421]:
def lie_alg(expr1, expr2, der, dic):
    coef = []
    for i in der:
        if expr1.has(i):
            coef.append(i)
    #list with the derivatives
    pol = sp.Poly(expr1, *der)
    pol = pol.coeffs()
    #list with the coefficients of the derivatives
    s = np.size(pol)
    for i in range(0, s):
        pol[i] = pol[i]*( coef[i].subs( coef[i], sp.diff( expr2 , dic[ coef[i] ]  ) ) )
        #this simulates v_1*v_2 with the derivative applied.
    return np.sum(pol)

In [422]:
#Kinda late to implement this but whatever. Didn't use it at all.
def commutator(expr1, expr2, der, dic):
    comm = lieg_alg(expr1, expr2, der, dic) - lieg_alg(expr2, expr1, der, dic)
    return comm

In [423]:
#Implementation of the first set of equations we have from part 1 of the results.
#And coefficient comparison
eq1 = np.array([0, 0, 0, 0, 0, 0])
eq2 = np.array([1, 1, 2, 2, 3, 3])
var1= np.array([x, t, y, t, z, t])
var2= np.array([t, x, t, y, t, z])
expr1 = []
for i in range(0, 6):
    expr1.append(eqs[ eq1[i] ].diff( var1[i] ) - eqs[ eq2[i] ].diff( var2[i]) )
    dicti = comp( expr1[i], lvar, lc, dicti)
    eqs = eqs.subs(dicti)

In [424]:
eqs

Matrix([
[a_0 + a_1*t + a_10*x*y + a_11*x*z + a_13*y*z + a_2*x + a_3*y + a_4*z + a_5*t**2 + a_5*x**2 + a_5*y**2 + a_5*z**2 + a_6*t*x + a_7*t*y + a_8*t*z],
[   a_1*x + a_10*t*y + a_11*t*z + a_2*t + 2*a_5*t*x + a_6*t**2/2 + a_6*x**2/2 + a_7*x*y + a_8*x*z + b_0 + b_12*y**2 + b_14*z**2 + b_3*y + b_4*z],
[    a_1*y + a_10*t*x + a_13*t*z + a_3*t + 2*a_5*t*y + a_6*x*y + a_7*t**2/2 + a_7*y**2/2 + a_8*y*z + c_0 + c_14*z**2 + c_2*x + c_4*z + c_9*x**2],
[    a_1*z + a_11*t*x + a_13*t*y + a_4*t + 2*a_5*t*z + a_6*x*z + a_7*y*z + a_8*t**2/2 + a_8*z**2/2 + d_0 + d_12*y**2 + d_2*x + d_3*y + d_9*x**2]])

In [425]:
#Repeat with the next given set
eq1 = np.array([1, 2, 1])
eq2 = np.array([2, 3, 3])
var1= np.array([y, z, z])
var2= np.array([x, y, x])
expr2 = []
for i in range(0, 3):
    expr2.append(eqs[ eq1[i] ].diff( var1[i] ) + eqs[ eq2[i] ].diff( var2[i]) )
    dicti = comp( expr2[i], lvar, lc, dicti)
    eqs = eqs.subs(dicti)

In [426]:
eqs

Matrix([
[          a_0 + a_1*t + a_2*x + a_3*y + a_4*z + a_5*t**2 + a_5*x**2 + a_5*y**2 + a_5*z**2 + a_6*t*x + a_7*t*y + a_8*t*z],
[a_1*x + a_2*t + 2*a_5*t*x + a_6*t**2/2 + a_6*x**2/2 - a_6*y**2/2 - a_6*z**2/2 + a_7*x*y + a_8*x*z + b_0 + b_3*y + b_4*z],
[a_1*y + a_3*t + 2*a_5*t*y + a_6*x*y + a_7*t**2/2 - a_7*x**2/2 + a_7*y**2/2 - a_7*z**2/2 + a_8*y*z - b_3*x + c_0 + c_4*z],
[a_1*z + a_4*t + 2*a_5*t*z + a_6*x*z + a_7*y*z + a_8*t**2/2 - a_8*x**2/2 - a_8*y**2/2 + a_8*z**2/2 - b_4*x - c_4*y + d_0]])

In [427]:
τ, ξ, γ, ζ = uncomb(eqs)

In [428]:
#double derivative equation yield 0 immediately 
#unnecessary

We will now proceed to find out the φ,χ,ψ,ω equations. With a polynomial Ansatz of the first order, which can be proved by derivating the other given equations.

 
 $$ 
 \\  ξ_{y} + φ_{w1} - χ_{w2}, 
 \\  τ_{z} - φ_{u1} + χ_{u2}, 
\\  - ξ_{z} - φ_{u2} + χ_{u1}, 
\\  - τ_{x} + φ_{u2} + χ_{u1}, 
\\  τ_{y} - φ_{w2} + χ_{w1}, 
\\  - γ_{z} - φ_{w2} - χ_{w1},
\\  ξ_{z} - τ_{x} + 2 φ_{u2}, 
\\  γ_{z} - τ_{y} + 2 φ_{w2}, 
\\  ξ_{z} + τ_{x} - 2 χ_{u1}, 
\\  - γ_{z} - τ_{y} - 2 χ_{w1}, 
\\  φ_{t} + φ_{z} + χ_{x} + ω_{y}, 
\\  - χ_{y} + ψ_{t} + ψ_{z} + ω_{x}, 
\\  φ_{x} + χ_{t} - χ_{z} - ψ_{y}, 
\\  φ_{y} + ψ_{x} + ω_{t} - ω_{z}  

In [429]:
#phi function
f1 = (-eqs[1].diff(z) + eqs[0].diff(x))/2
f2 = (-eqs[2].diff(z) + eqs[0].diff(y))/2
phi = f1*u2 + f2*w2 
phi
#k_0 function of u2, w2!, t,x,y,z also

u2*(a_2/2 + a_5*x + a_6*t/2 + a_6*z/2 - a_8*x/2 - b_4/2) + w2*(a_3/2 + a_5*y + a_7*t/2 + a_7*z/2 - a_8*y/2 - c_4/2)

In [430]:
#chi function
g1 = ( eqs[1].diff(z) + eqs[0].diff(x))/2
g2 = (-eqs[2].diff(z) - eqs[0].diff(y))/2
chi = g1*u1 + g2*w1 
chi               
#k_1  function of u1, w1, t,x,y,z

u1*(a_2/2 + a_5*x + a_6*t/2 - a_6*z/2 + a_8*x/2 + b_4/2) + w1*(-a_3/2 - a_5*y - a_7*t/2 + a_7*z/2 - a_8*y/2 - c_4/2)

Ansatz: $u_1, u_2, w_1, w_2$ are polynomials of the first order

In [431]:
#Ansatz of linearity for w1 and u1
#But are t,x,y,z also linear in the product with them? or of higher order?
#we know that φ,χ,ψ,ω musst satisfy the wave equation as well. 1st order does, but so does 2nd order as well...
phi = phi + (f[1]*t + f[2]*x + f[3]*y + f[4]*z + f[5])*w1 
phi = phi + (f[6]*t + f[7]*x + f[8]*y + f[9]*z + f[10])*u1 + f[11]*β1
phi = sp.collect(phi, [t, x, y, z])

In [432]:
phi

f_11*β_1(t, x, y, z) + u1*(f_10 + f_6*t + f_7*x + f_8*y + f_9*z) + u2*(a_2/2 + a_6*t/2 + a_6*z/2 - b_4/2 + x*(a_5 - a_8/2)) + w1*(f_1*t + f_2*x + f_3*y + f_4*z + f_5) + w2*(a_3/2 + a_7*t/2 + a_7*z/2 - c_4/2 + y*(a_5 - a_8/2))

In [433]:
#psi - w_2 relation ii)
rel1 = eqs[0].diff(z) - sp.Derivative(φ, u1) + sp.Derivative(χ, u2)
rel1

a_4 + 2*a_5*z + a_8*t - Derivative(φ, u1) + Derivative(χ, u2)

In [434]:
rel2 = eqs[1].diff(y) + sp.Derivative(φ, w1) - sp.Derivative(χ, w2)
rel2

-a_6*y + a_7*x + b_3 + Derivative(φ, w1) - Derivative(χ, w2)

In [435]:
#find the other chi terms
p1 = -( eqs[0].diff(z) - sp.diff(phi, u1) )*u2
p2 = ( eqs[1].diff(y) + sp.diff(phi, w1) )*w2
chi = chi + p1 + p2 + f[12]*β2
chi = sp.collect(chi, [t, x, y, z])

psi = -sp.diff(phi, w2)*u2 + sp.diff(phi, u2)*w2 - sp.diff(phi, w1)*u1 + sp.diff(phi, u1)*w1
psi = sp.collect(psi, [t, x, y, z]) + f[13]*β3

omg = sp.diff(chi, u2)*w2 - sp.diff(chi, w2)*u2 + sp.diff(chi, u1)*w1 - sp.diff(chi, w1)*u1
omg = sp.collect(omg, [t, x, y, z]) + f[14]*β4

In [436]:
eqs

Matrix([
[          a_0 + a_1*t + a_2*x + a_3*y + a_4*z + a_5*t**2 + a_5*x**2 + a_5*y**2 + a_5*z**2 + a_6*t*x + a_7*t*y + a_8*t*z],
[a_1*x + a_2*t + 2*a_5*t*x + a_6*t**2/2 + a_6*x**2/2 - a_6*y**2/2 - a_6*z**2/2 + a_7*x*y + a_8*x*z + b_0 + b_3*y + b_4*z],
[a_1*y + a_3*t + 2*a_5*t*y + a_6*x*y + a_7*t**2/2 - a_7*x**2/2 + a_7*y**2/2 - a_7*z**2/2 + a_8*y*z - b_3*x + c_0 + c_4*z],
[a_1*z + a_4*t + 2*a_5*t*z + a_6*x*z + a_7*y*z + a_8*t**2/2 - a_8*x**2/2 - a_8*y**2/2 + a_8*z**2/2 - b_4*x - c_4*y + d_0]])

In [437]:
sp.Matrix([phi, chi, psi, omg])

Matrix([
[                                                    f_11*β_1(t, x, y, z) + u1*(f_10 + f_6*t + f_7*x + f_8*y + f_9*z) + u2*(a_2/2 + a_6*t/2 + a_6*z/2 - b_4/2 + x*(a_5 - a_8/2)) + w1*(f_1*t + f_2*x + f_3*y + f_4*z + f_5) + w2*(a_3/2 + a_7*t/2 + a_7*z/2 - c_4/2 + y*(a_5 - a_8/2))],
[f_12*β_2(t, x, y, z) + u1*(a_2/2 + a_6*t/2 - a_6*z/2 + b_4/2 + x*(a_5 + a_8/2)) + u2*(-a_4 + f_10 + f_7*x + f_8*y + t*(-a_8 + f_6) + z*(-2*a_5 + f_9)) + w1*(-a_3/2 - a_7*t/2 + a_7*z/2 - c_4/2 + y*(-a_5 - a_8/2)) + w2*(b_3 + f_1*t + f_4*z + f_5 + x*(a_7 + f_2) + y*(-a_6 + f_3))],
[                                                  f_13*β_3(t, x, y, z) - u1*(f_1*t + f_2*x + f_3*y + f_4*z + f_5) + u2*(-a_3/2 - a_7*t/2 - a_7*z/2 + c_4/2 + y*(-a_5 + a_8/2)) + w1*(f_10 + f_6*t + f_7*x + f_8*y + f_9*z) + w2*(a_2/2 + a_6*t/2 + a_6*z/2 - b_4/2 + x*(a_5 - a_8/2))],
[f_14*β_4(t, x, y, z) - u1*(-a_3/2 - a_7*t/2 + a_7*z/2 - c_4/2 + y*(-a_5 - a_8/2)) - u2*(b_3 + f_1*t + f_4*z + f_5 + x*(a_7 + f_2) + y*(-a_6 + f_3))

In [438]:
fsys = [
(-chi.diff(y) + psi.diff(t) + psi.diff(z) + omg.diff(x)   ).subs({f[11]:0, f[12]:0, f[13]:0, f[14]:0  }),
( phi.diff(x) + chi.diff(t) - chi.diff(z) - psi.diff(y)   ).subs({f[11]:0, f[12]:0, f[13]:0, f[14]:0  }),
( phi.diff(y) + psi.diff(x) + omg.diff(t) - omg.diff(z)   ).subs({f[11]:0, f[12]:0, f[13]:0, f[14]:0  }),
( phi.diff(t) + phi.diff(z) + chi.diff(x) + omg.diff(y)   ).subs({f[11]:0, f[12]:0, f[13]:0, f[14]:0  })
]

In [439]:
sp.Matrix(fsys)

Matrix([
[   a_6*w2 - a_7*u2 - f_1*u1 - f_4*u1 + f_6*w1 + f_7*w2 - f_8*u2 + f_9*w1 - u2*(a_7 + f_2) - w1*(-a_5 - a_8/2) + w1*(a_5 + a_8/2) - w2*(-a_6 + f_3)],
[a_6*u1 - a_7*w1 + f_1*w2 + f_2*w1 + f_3*u1 - f_4*w2 + f_7*u1 - f_8*w1 - u2*(-2*a_5 + f_9) - u2*(-a_5 + a_8/2) + u2*(a_5 - a_8/2) + u2*(-a_8 + f_6)],
[                  a_6*w1 + a_7*u1 - f_1*u2 - f_2*u1 + f_3*w1 + f_4*u2 + f_7*w1 + f_8*u1 - w2*(-2*a_5 + f_9) + 2*w2*(a_5 - a_8/2) + w2*(-a_8 + f_6)],
[   a_6*u2 + a_7*w2 + f_1*w1 + f_4*w1 + f_6*u1 + f_7*u2 + f_8*w2 + f_9*u1 - u1*(-a_5 - a_8/2) + u1*(a_5 + a_8/2) - u2*(-a_6 + f_3) + w2*(a_7 + f_2)]])

In [440]:
sys_solve = []
for i in lwlf:
    temp = sp.simplify(sp.Matrix(fsys))
    for j in lwlf:
        if i != j:
            temp = temp.subs({j:0})
    temp = temp.subs({i:1})
    for k in range(0, 4):
        sys_solve.append(temp[k])


In [441]:
sp.Matrix(sys_solve)

Matrix([
[               -f_1 - f_4],
[          a_6 + f_3 + f_7],
[          a_7 - f_2 + f_8],
[  2*a_5 + a_8 + f_6 + f_9],
[       -2*a_7 - f_2 - f_8],
[4*a_5 - 2*a_8 + f_6 - f_9],
[               -f_1 + f_4],
[        2*a_6 - f_3 + f_7],
[  2*a_5 + a_8 + f_6 + f_9],
[         -a_7 + f_2 - f_8],
[          a_6 + f_3 + f_7],
[                f_1 + f_4],
[        2*a_6 - f_3 + f_7],
[                f_1 - f_4],
[4*a_5 - 2*a_8 + f_6 - f_9],
[        2*a_7 + f_2 + f_8]])

In [442]:
find = []
for sym in lc0+f:
    if sp.Matrix(sys_solve).has(sym):
        find.append(sym)

In [443]:
find

[a_5, a_6, a_7, a_8, f_1, f_2, f_3, f_4, f_6, f_7, f_8, f_9]

In [444]:
sol = sp.linsolve(sys_solve, find)
sol

FiniteSet((-3*f_6/8 - f_9/8, -2*f_7/3, -2*f_8/3, -f_6/4 - 3*f_9/4, 0, f_8/3, -f_7/3, 0, f_6, f_7, f_8, f_9))

In [445]:
dic_sys = dict(zip(find, *sol))

In [446]:
dic_sys

{a_5: -3*f_6/8 - f_9/8,
 a_6: -2*f_7/3,
 a_7: -2*f_8/3,
 a_8: -f_6/4 - 3*f_9/4,
 f_1: 0,
 f_2: f_8/3,
 f_3: -f_7/3,
 f_4: 0,
 f_6: f_6,
 f_7: f_7,
 f_8: f_8,
 f_9: f_9}

In [447]:
final_eqs = sp.Matrix([eqs[0], eqs[1], eqs[2], eqs[3], phi, chi, psi, omg]).subs(dic_sys)
final_eqs

Matrix([
[                                                                        a_0 + a_1*t + a_2*x + a_3*y + a_4*z - 2*f_7*t*x/3 - 2*f_8*t*y/3 + t**2*(-3*f_6/8 - f_9/8) + t*z*(-f_6/4 - 3*f_9/4) + x**2*(-3*f_6/8 - f_9/8) + y**2*(-3*f_6/8 - f_9/8) + z**2*(-3*f_6/8 - f_9/8)],
[                                                                                                               a_1*x + a_2*t + b_0 + b_3*y + b_4*z - f_7*t**2/3 - f_7*x**2/3 + f_7*y**2/3 + f_7*z**2/3 - 2*f_8*x*y/3 + 2*t*x*(-3*f_6/8 - f_9/8) + x*z*(-f_6/4 - 3*f_9/4)],
[                                                                                                               a_1*y + a_3*t - b_3*x + c_0 + c_4*z - 2*f_7*x*y/3 - f_8*t**2/3 + f_8*x**2/3 - f_8*y**2/3 + f_8*z**2/3 + 2*t*y*(-3*f_6/8 - f_9/8) + y*z*(-f_6/4 - 3*f_9/4)],
[                                                              a_1*z + a_4*t - b_4*x - c_4*y + d_0 - 2*f_7*x*z/3 - 2*f_8*y*z/3 + t**2*(-f_6/4 - 3*f_9/4)/2 + 2*t*z*(-3*f_6/8 - f_9/8) - x**

In [448]:
lc_f = []
for i in lc0+lc1+lc2+lc3+f:
    if final_eqs.has(i):
        lc_f.append(i)
np.size(lc_f)

21

In [449]:
#Lets generate all the possible vector fields
v_fields = []
sl = np.size(lc_f)
for i in range(0, sl):
    arr = sp.Matrix(final_eqs.copy())
    for j in range(0, sl):
        if i != j:
            arr = arr.subs(lc_f[j], 0)
    temp = 0
    #could probably do a np.sum( arr ) and be done, but whatever
    for k in range(0, 8):
        temp = temp + arr[k].subs(lc_f[i], 1)*der[k]
    v_fields.append(temp)
np.size(v_fields)

21

$[v_1, v_2] = v_1 \cdot v_2 - v_2 \cdot v_1 = c \cdot v_j$ or $= 0$

In [450]:
enum = range(0, sl)

In [451]:
sp.Matrix([enum, v_fields]).T

Matrix([
[ 0,                                                                                                                                                                                                                                                                                                                            δ_t],
[ 1,                                                                                                                                                                                                                                                                                                  t*δ_t + x*δ_x + y*δ_y + z*δ_z],
[ 2,                                                                                                                                                                                                                                                                  t*δ_x + u1*δ_u2/2 + u2*δ_u1/2 + w1*δ_w2/2 + w2*δ_w1/2 + x*δ_t],
[ 3,         

In [452]:
lvar_f = lvar + [u1, u2, w1, w2]
dic_der = dict(list(zip(der, lvar_f)))
der

[δ_t, δ_x, δ_y, δ_z, δ_u1, δ_u2, δ_w1, δ_w2]

In [453]:
import time 

In [454]:
#rewritten the original lie_alg() function to include the case of 0, since this can happen now as well
def lie_alg_t(expr1, expr2, der, dic):
    if expr1 == 0 or expr2 == 0:
        return 0
    else: 
        terms = []
        for i in der:
            if expr1.has(i):
                terms.append(i)
        #list with the derivatives
        pol = sp.Poly(expr1, *der)
        pol = pol.coeffs()
        #list with the coefficients of the derivatives
        s = np.size(pol)
        for i in range(0, s):
            pol[i] = pol[i]*( terms[i].subs( terms[i], sp.diff( expr2 , dic[ terms[i] ]  ) ) )
            #this simulates v_1*v_2 with the derivative applied.
        return np.sum(pol)

Skew symmetry:
$$ [u, w] = - [w, u]$$

Bilinearity:
$$ [qv + pu, w] = q[v, w] + p[u,w]$$

Jacobi identity:
$$[u, [v, w]] + [w, [u, v]] + [v, [w, u]] = 0$$
$$a = [v, w]$$
$$b = [u, v]$$
$$c = [w, u]$$

In [455]:
#Check 
check = input("Run the checks? y/n")

if check == "y":
    t0 = time.time()

    #SKEW SYMMETRY
    skew = (np.zeros(sl**2).reshape(sl, sl)).tolist()
    for i in range(0, sl):
        for j in range(0, sl):
            skew[i][j] = lie_alg(v_fields[i], v_fields[j], der, dic_der ) - lie_alg(v_fields[j], v_fields[i], der, dic_der )

    for i in range(0, sl):
        for j in range(0, i+1):
            if skew[i][j] != -skew[j][i] :
                print("For i: ", i, "and j: ", j)
                print("Not fulifilled")

    for i in range(0, sl):
        if skew[i][i] != 0:
            print("You fuckit up")
    #PASSED!

    t1 = time.time()

    #Bilinearity
    for i in range(0, sl):
        v = v_fields[i]
        for j in range(0, sl):
            u = v_fields[j]
            for k in range(0, sl):
                w = v_fields[k]

                bil1 = sp.expand( lie_alg_t( sp.expand(q*v+p*u), w, der, dic_der ) - lie_alg_t( w, sp.expand(q*v+p*u), der, dic_der ) )
                bil2 = sp.expand(  q*( lie_alg_t( v, w, der, dic_der ) - lie_alg_t( w, v, der, dic_der )  )  )
                bil3 = sp.expand(  p*( lie_alg_t( u, w, der, dic_der ) - lie_alg_t( w, u, der, dic_der )  )  )
                if sp.expand(bil1 - bil2 - bil3) != 0:
                    print("Bilinearity not satisfied")
                    print(i, j, k)
    t2 = time.time()

    #JACOBI IDENTITY

    for i in range(0, sl):
        u = v_fields[i]
        for j in range(0, sl):
            v = v_fields[j]
            b  = sp.expand(lie_alg_t(u, v, der, dic_der)) - sp.expand(lie_alg_t(v, u, der, dic_der))
            for k in range(0, sl):
                w = v_fields[k]
                a  = sp.expand(lie_alg_t(v, w, der, dic_der)) - sp.expand(lie_alg_t(w, v, der, dic_der))

                c  = sp.expand(lie_alg_t(w, u, der, dic_der)) - sp.expand(lie_alg_t(u, w, der, dic_der))
                e1 = sp.expand(lie_alg_t(u, a ,der, dic_der)) - sp.expand(lie_alg_t(a, u ,der, dic_der))
                e2 = sp.expand(lie_alg_t(w, b ,der, dic_der)) - sp.expand(lie_alg_t(b, w ,der, dic_der))
                e3 = sp.expand(lie_alg_t(v, c ,der, dic_der)) - sp.expand(lie_alg_t(c, v ,der, dic_der))
                if sp.expand(e1 + e2 + e3) !=0:
                    print("Jacobi Identity not fuliffiled")
                    print(i, j, k)

    #PASSED!
    #commented because it takes too long to evaluated

    t3 = time.time()

    print("Skew symmetry time:", t1-t0)
    print("Bilinearity time:", t2-t1)
    print("Jacobi identity time:", t3- t2)

In [456]:
#Rearrange the v_fields to match the ones from the thesis
sp.Matrix([enum, v_fields]).T

Matrix([
[ 0,                                                                                                                                                                                                                                                                                                                            δ_t],
[ 1,                                                                                                                                                                                                                                                                                                  t*δ_t + x*δ_x + y*δ_y + z*δ_z],
[ 2,                                                                                                                                                                                                                                                                  t*δ_x + u1*δ_u2/2 + u2*δ_u1/2 + w1*δ_w2/2 + w2*δ_w1/2 + x*δ_t],
[ 3,         

In [457]:
nvf = v_fields.copy()
nvf[1] = v_fields[5]
nvf[2] = v_fields[8]
nvf[3] = v_fields[10]
nvf[4] = v_fields[1]
nvf[5] = v_fields[2]
nvf[6] = v_fields[3]
nvf[7] = v_fields[4] + v_fields[16]/2
nvf[8] = v_fields[6] - v_fields[11]/2
nvf[9] = v_fields[7]
nvf[10] = v_fields[9]
nvf[11] = v_fields[16]
nvf[12] = -v_fields[11]
nvf[13] = v_fields[12]
nvf[14] = v_fields[13]
nvf[15] = v_fields[14]
nvf[16] = v_fields[15]


In [458]:
nvf[13] = (sp.expand(nvf[13])*8).collect(der)
nvf[14] = (sp.expand(nvf[14])*(-3)).collect(der)
nvf[15] = (sp.expand(nvf[15])*(-3)).collect(der)
nvf[16] = (sp.expand(nvf[16])*8).collect(der)

In [459]:
temp1 = ( 3*(sp.expand(nvf[13]) - (sp.expand(nvf[16])/3) )/8 ).collect(der)
temp1

-2*t*x*δ_x - 2*t*y*δ_y - 2*t*z*δ_z + δ_t*(-t**2 - x**2 - y**2 - z**2) + δ_u1*(3*t*u1 - u1*z - u2*x - w2*y) + δ_u2*(3*t*u2 - u1*x + u2*z + w1*y) + δ_w1*(3*t*w1 + u2*y - w1*z - w2*x) + δ_w2*(3*t*w2 - u1*y - w1*x + w2*z)

In [460]:
temp2 = ( 3*( (sp.expand(nvf[13]))/3 - (sp.expand(nvf[16]))  )/8  ).collect(der)
temp2

2*t*z*δ_t + 2*x*z*δ_x + 2*y*z*δ_y + δ_u1*(t*u1 - 3*u1*z - u2*x - w2*y) + δ_u2*(-t*u2 + u1*x - 3*u2*z - w1*y) + δ_w1*(t*w1 + u2*y - 3*w1*z - w2*x) + δ_w2*(-t*w2 + u1*y + w1*x - 3*w2*z) + δ_z*(t**2 - x**2 - y**2 + z**2)

In [461]:
nvf[13] = (-1*sp.expand(temp1)).collect(der)
nvf[16] = temp2

In [462]:
sp.Matrix([enum, nvf]).T

Matrix([
[ 0,                                                                                                                                                                                                                         δ_t],
[ 1,                                                                                                                                                                                                                         δ_x],
[ 2,                                                                                                                                                                                                                         δ_y],
[ 3,                                                                                                                                                                                                                         δ_z],
[ 4,                                                                               

In [772]:
P = sp.Matrix([nvf[0], nvf[1], nvf[2], nvf[3]])        # 0-3

D = ( (nvf[4]) - (3*nvf[11]/2) )                           # Just one

M = sp.Matrix([
                [ 0      ,nvf[5]  , nvf[6]  , nvf[7] ],   #
                [-nvf[5] , 0      , nvf[8]  , nvf[9] ],   #This is 0-3 and 0-3
                [-nvf[6] , -nvf[8], 0       , nvf[10]],   #
                [-nvf[7] , -nvf[9], -nvf[10], 0      ]    #
                ])                                     # Vorzeichen? wegen x-y, x-z, y-z

K = sp.Matrix([-nvf[13], nvf[14], nvf[15], nvf[16]])    # 0 -3

U1 = nvf[11].copy() #scale
U2 = nvf[12].copy() #phase

h = sp.eye(6, 6)
h[0, 0] = -1
h[1, 1] = -1
h

Matrix([
[-1,  0, 0, 0, 0, 0],
[ 0, -1, 0, 0, 0, 0],
[ 0,  0, 1, 0, 0, 0],
[ 0,  0, 0, 1, 0, 0],
[ 0,  0, 0, 0, 1, 0],
[ 0,  0, 0, 0, 0, 1]])

I'm not quite sure how to make this J matrix...
From the paper it is like this:
$$\begin{bmatrix}
0 & D & P_m - K_m \\
-D & 0 & P_m + K_m \\ 
-- & -- & L_{ab}   
\end{bmatrix}  $$
I think doing it equivalently is the rational thing...

In [792]:
#Based on the script
J0 = sp.zeros(6, 6)

J0[0, 1] =  D
J0[1, 0] = -D

for i in range(2, 6):
    J0[0, i] =  (P[i-2] - K[i-2])/2
    J0[1, i] =  (P[i-2] + K[i-2])/2

    J0[i, 0] = -(P[i-2] - K[i-2])/2
    J0[i, 1] = -(P[i-2] + K[i-2])/2

    for j in range(2, 6):
        J0[i, j] = M[i-2, j-2]

J0.is_anti_symmetric()

True

In [773]:
#Based on Olaf's drawing
J1 = sp.zeros(6, 6)

for i in range(0, 4):
    J1[0, 1+i] = ( (K[i]) + P[i])/2

J1[0, 5] = D

for i in range(0, 4):
    for j in range(0, 4):
        J1[1+i, 1+j] = M[i, j]

for i in range(0, 4):
    J1[1+i, 5] = ( (K[i]) - P[i])/2

for i in range(0, 5):
    J1[i, 0] = -J1[0, i]

for i in range(0, 5):
    J1[5, i] = -J1[i, 5]
    
J1.is_anti_symmetric()

True

In [793]:
s = [+1, +1, -1, -1]
J = J0
flag = True
for a in range(0, 6):
    for b in range(0, 6):
        v0 = J[a, b]
        for c in range(0, 6):
            for d in range(0, 6):
                v1 = J[c, d] 
                #left hand of the equation
                temp1 = sp.expand(lie_alg_t(v0, v1, der, dic_der) - lie_alg_t(v1, v0, der, dic_der))
                temp2 = sp.expand( s[0]*h[a,d]*J[b, c] + s[1]*h[b,c]*J[a,d] + s[2]*h[a,c]*J[b,d] + s[3]*h[b,d]*J[a,c])
                if temp1 != temp2:
                    print(a, b, c, d)
                    flag = False
print(flag)

0 1 0 2
0 1 0 3
0 1 0 4
0 1 0 5
0 1 2 0
0 1 3 0
0 1 4 0
0 1 5 0
0 2 0 1
0 2 0 3
0 2 0 4
0 2 0 5
0 2 1 0
0 2 1 2
0 2 1 3
0 2 1 4
0 2 1 5
0 2 2 1
0 2 2 3
0 2 2 4
0 2 2 5
0 2 3 0
0 2 3 1
0 2 3 2
0 2 4 0
0 2 4 1
0 2 4 2
0 2 5 0
0 2 5 1
0 2 5 2
0 3 0 1
0 3 0 2
0 3 0 4
0 3 0 5
0 3 1 0
0 3 1 2
0 3 1 3
0 3 2 0
0 3 2 1
0 3 2 3
0 3 3 1
0 3 3 2
0 3 3 4
0 3 3 5
0 3 4 0
0 3 4 3
0 3 5 0
0 3 5 3
0 4 0 1
0 4 0 2
0 4 0 3
0 4 0 5
0 4 1 0
0 4 1 2
0 4 1 4
0 4 2 0
0 4 2 1
0 4 2 4
0 4 3 0
0 4 3 4
0 4 4 1
0 4 4 2
0 4 4 3
0 4 4 5
0 4 5 0
0 4 5 4
0 5 0 1
0 5 0 2
0 5 0 3
0 5 0 4
0 5 1 0
0 5 1 2
0 5 1 5
0 5 2 0
0 5 2 1
0 5 2 5
0 5 3 0
0 5 3 5
0 5 4 0
0 5 4 5
0 5 5 1
0 5 5 2
0 5 5 3
0 5 5 4
1 0 0 2
1 0 0 3
1 0 0 4
1 0 0 5
1 0 2 0
1 0 3 0
1 0 4 0
1 0 5 0
1 2 0 2
1 2 0 3
1 2 0 4
1 2 0 5
1 2 1 3
1 2 1 4
1 2 1 5
1 2 2 0
1 2 2 3
1 2 2 4
1 2 2 5
1 2 3 0
1 2 3 1
1 2 3 2
1 2 4 0
1 2 4 1
1 2 4 2
1 2 5 0
1 2 5 1
1 2 5 2
1 3 0 2
1 3 0 3
1 3 1 2
1 3 2 0
1 3 2 1
1 3 2 3
1 3 3 0
1 3 3 2
1 3 3 4
1 3 3 5
1 3 4 3
1 3 5 3
1 4 0 2


KeyboardInterrupt: 

In [784]:
J = J0
a = 5
b = 5
c = 5
d = 3
v0 = J[ a, b]
v1 = J[ c, d] 

In [785]:
sp.expand(lie_alg_t(v0, v1, der, dic_der) - lie_alg_t(v1, v0, der, dic_der))

0

In [787]:
sp.expand( s[0]*h[a,d]*J[b, c] + s[1]*h[b,c]*J[a,d] - s[2]*h[a,c]*J[b,d] + s[3]*h[b,d]*J[a,c])

0

In [466]:
M

Matrix([
[                                                             0,  t*δ_x + u1*δ_u2/2 + u2*δ_u1/2 + w1*δ_w2/2 + w2*δ_w1/2 + x*δ_t,  t*δ_y + u1*δ_w2/2 - u2*δ_w1/2 - w1*δ_u2/2 + w2*δ_u1/2 + y*δ_t, t*δ_z + u1*δ_u1/2 - u2*δ_u2/2 + w1*δ_w1/2 - w2*δ_w2/2 + z*δ_t],
[-t*δ_x - u1*δ_u2/2 - u2*δ_u1/2 - w1*δ_w2/2 - w2*δ_w1/2 - x*δ_t,                                                              0,  u1*δ_w1/2 - u2*δ_w2/2 - w1*δ_u1/2 + w2*δ_u2/2 - x*δ_y + y*δ_x, u1*δ_u2/2 - u2*δ_u1/2 + w1*δ_w2/2 - w2*δ_w1/2 - x*δ_z + z*δ_x],
[-t*δ_y - u1*δ_w2/2 + u2*δ_w1/2 + w1*δ_u2/2 - w2*δ_u1/2 - y*δ_t, -u1*δ_w1/2 + u2*δ_w2/2 + w1*δ_u1/2 - w2*δ_u2/2 + x*δ_y - y*δ_x,                                                              0, u1*δ_w2/2 + u2*δ_w1/2 - w1*δ_u2/2 - w2*δ_u1/2 - y*δ_z + z*δ_y],
[-t*δ_z - u1*δ_u1/2 + u2*δ_u2/2 - w1*δ_w1/2 + w2*δ_w2/2 - z*δ_t, -u1*δ_u2/2 + u2*δ_u1/2 - w1*δ_w2/2 + w2*δ_w1/2 + x*δ_z - z*δ_x, -u1*δ_w2/2 - u2*δ_w1/2 + w1*δ_u2/2 + w2*δ_u1/2 + y*δ_z - z*δ_y,                            

In [467]:
#[M_μν, M_ρσ]
l9 = []
for i in range(0, 4):
    l9.append([])
    for j in range(0,4):
        v0 = M[i, j]
        l9[i].append([])
        for k in range(0, 4):
            for l in range(0, 4):
                v1 = M[k, l]
                temp = sp.expand(lie_alg_t(v0, v1, der, dic_der) - lie_alg_t(v1, v0, der, dic_der))
                l9[i][j].append(temp.collect(der))

In [537]:
M

Matrix([
[                                                             0,  t*δ_x + u1*δ_u2/2 + u2*δ_u1/2 + w1*δ_w2/2 + w2*δ_w1/2 + x*δ_t,  t*δ_y + u1*δ_w2/2 - u2*δ_w1/2 - w1*δ_u2/2 + w2*δ_u1/2 + y*δ_t, t*δ_z + u1*δ_u1/2 - u2*δ_u2/2 + w1*δ_w1/2 - w2*δ_w2/2 + z*δ_t],
[-t*δ_x - u1*δ_u2/2 - u2*δ_u1/2 - w1*δ_w2/2 - w2*δ_w1/2 - x*δ_t,                                                              0,  u1*δ_w1/2 - u2*δ_w2/2 - w1*δ_u1/2 + w2*δ_u2/2 - x*δ_y + y*δ_x, u1*δ_u2/2 - u2*δ_u1/2 + w1*δ_w2/2 - w2*δ_w1/2 - x*δ_z + z*δ_x],
[-t*δ_y - u1*δ_w2/2 + u2*δ_w1/2 + w1*δ_u2/2 - w2*δ_u1/2 - y*δ_t, -u1*δ_w1/2 + u2*δ_w2/2 + w1*δ_u1/2 - w2*δ_u2/2 + x*δ_y - y*δ_x,                                                              0, u1*δ_w2/2 + u2*δ_w1/2 - w1*δ_u2/2 - w2*δ_u1/2 - y*δ_z + z*δ_y],
[-t*δ_z - u1*δ_u1/2 + u2*δ_u2/2 - w1*δ_w1/2 + w2*δ_w2/2 - z*δ_t, -u1*δ_u2/2 + u2*δ_u1/2 - w1*δ_w2/2 + w2*δ_w1/2 + x*δ_z - z*δ_x, -u1*δ_w2/2 - u2*δ_w1/2 + w1*δ_u2/2 + w2*δ_u1/2 + y*δ_z - z*δ_y,                            

In [468]:
sp.Matrix(l9[2][3]).reshape(4, 4)

Matrix([
[                                                             0,                                                              0, t*δ_z + u1*δ_u1/2 - u2*δ_u2/2 + w1*δ_w1/2 - w2*δ_w2/2 + z*δ_t, -t*δ_y - u1*δ_w2/2 + u2*δ_w1/2 + w1*δ_u2/2 - w2*δ_u1/2 - y*δ_t],
[                                                             0,                                                              0, u1*δ_u2/2 - u2*δ_u1/2 + w1*δ_w2/2 - w2*δ_w1/2 - x*δ_z + z*δ_x, -u1*δ_w1/2 + u2*δ_w2/2 + w1*δ_u1/2 - w2*δ_u2/2 + x*δ_y - y*δ_x],
[-t*δ_z - u1*δ_u1/2 + u2*δ_u2/2 - w1*δ_w1/2 + w2*δ_w2/2 - z*δ_t, -u1*δ_u2/2 + u2*δ_u1/2 - w1*δ_w2/2 + w2*δ_w1/2 + x*δ_z - z*δ_x,                                                             0,                                                              0],
[ t*δ_y + u1*δ_w2/2 - u2*δ_w1/2 - w1*δ_u2/2 + w2*δ_u1/2 + y*δ_t,  u1*δ_w1/2 - u2*δ_w2/2 - w1*δ_u1/2 + w2*δ_u2/2 - x*δ_y + y*δ_x,                                                             0,                             

In [552]:
for i in range(0, 4):
    for j in range(0, 4):
        if sp.Matrix(l9[2][3]).reshape(4, 4)[0,2] == M[i, j] :
            print("plus variant", i, j)


plus variant 0 3


In [469]:
v0 = M[2, 3]
v1 = M[2, 0]
sp.expand(lie_alg_t(v0, v1, der, dic_der) - lie_alg_t(v1, v0, der, dic_der))

-t*δ_z - u1*δ_u1/2 + u2*δ_u2/2 - w1*δ_w1/2 + w2*δ_w2/2 - z*δ_t

In [470]:
# [M, U_i]
l8 = []
#v0 = U1
v0 = U2
for i in range(0, 4):
    for j in range(0,4):
        v1 = M[i, j]
        temp = -sp.expand(lie_alg_t(v0, v1, der, dic_der) - lie_alg_t(v1, v0, der, dic_der))
        l8.append(temp.collect(der))
sp.Matrix(l8).reshape(4, 4)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [574]:
#[M, K]
l7 = []
for i in range(0,4):
    v0 = K[i]
    l7.append([])
    for j in range(0,4):
        for k in range(0, 4):
            v1 = M[j, k]
            temp = -sp.expand(lie_alg_t(v0, v1, der, dic_der) - lie_alg_t(v1, v0, der, dic_der))
            l7[i].append(temp.collect(der))

In [575]:
#Elements of the main diagonal are 0 in all cases
sp.Matrix(l7[0]).reshape(4, 4)

Matrix([
[                                                                                                                                                                                                                        0, -2*t*x*δ_t - 2*x*y*δ_y - 2*x*z*δ_z + δ_u1*(-t*u2 + 3*u1*x - u2*z - w1*y) + δ_u2*(-t*u1 + u1*z + 3*u2*x + w2*y) + δ_w1*(-t*w2 + u1*y + 3*w1*x - w2*z) + δ_w2*(-t*w1 - u2*y + w1*z + 3*w2*x) + δ_x*(-t**2 - x**2 + y**2 + z**2), -2*t*y*δ_t - 2*x*y*δ_x - 2*y*z*δ_z + δ_u1*(-t*w2 + 3*u1*y + w1*x - w2*z) + δ_u2*(t*w1 + 3*u2*y - w1*z - w2*x) + δ_w1*(t*u2 - u1*x + u2*z + 3*w1*y) + δ_w2*(-t*u1 + u1*z + u2*x + 3*w2*y) + δ_y*(-t**2 + x**2 - y**2 + z**2), -2*t*z*δ_t - 2*x*z*δ_x - 2*y*z*δ_y + δ_u1*(-t*u1 + 3*u1*z + u2*x + w2*y) + δ_u2*(t*u2 - u1*x + 3*u2*z + w1*y) + δ_w1*(-t*w1 - u2*y + 3*w1*z + w2*x) + δ_w2*(t*w2 - u1*y - w1*x + 3*w2*z) + δ_z*(-t**2 + x**2 + y**2 - z**2)],
[  2*t*x*δ_t + 2*x*y*δ_y + 2*x*z*δ_z + δ_u1*(t*u2 - 3*u1*x + u2*z + w1*y) + δ_u2*(t*u1 - u1*z - 3*u2*x - 

In [609]:
sp.Matrix(l7[1]).reshape(4, 4)

Matrix([
[                                                                                                                                                                                                                        0, 2*t*x*δ_x + 2*t*y*δ_y + 2*t*z*δ_z + δ_t*(t**2 + x**2 + y**2 + z**2) + δ_u1*(-3*t*u1 + u1*z + u2*x + w2*y) + δ_u2*(-3*t*u2 + u1*x - u2*z - w1*y) + δ_w1*(-3*t*w1 - u2*y + w1*z + w2*x) + δ_w2*(-3*t*w2 + u1*y + w1*x - w2*z),                                                                                                                                                                                                                         0,                                                                                                                                                                                                                         0],
[-2*t*x*δ_x - 2*t*y*δ_y - 2*t*z*δ_z + δ_t*(-t**2 - x**2 - y**2 - z**2) + δ_u1*(3*t*u1 - u1*z - u2*x - w2*y) + δ

In [577]:
sp.Matrix(l7[2]).reshape(4, 4)

Matrix([
[                                                                                                                                                                                                                        0,                                                                                                                                                                                                                       0,   2*t*x*δ_x + 2*t*y*δ_y + 2*t*z*δ_z + δ_t*(t**2 + x**2 + y**2 + z**2) + δ_u1*(-3*t*u1 + u1*z + u2*x + w2*y) + δ_u2*(-3*t*u2 + u1*x - u2*z - w1*y) + δ_w1*(-3*t*w1 - u2*y + w1*z + w2*x) + δ_w2*(-3*t*w2 + u1*y + w1*x - w2*z),                                                                                                                                                                                                                         0],
[                                                                                                              

In [578]:
sp.Matrix(l7[3]).reshape(4, 4)

Matrix([
[                                                                                                                                                                                                                        0,                                                                                                                                                                                                                       0,                                                                                                                                                                                                                         0,   2*t*x*δ_x + 2*t*y*δ_y + 2*t*z*δ_z + δ_t*(t**2 + x**2 + y**2 + z**2) + δ_u1*(-3*t*u1 + u1*z + u2*x + w2*y) + δ_u2*(-3*t*u2 + u1*x - u2*z - w1*y) + δ_w1*(-3*t*w1 - u2*y + w1*z + w2*x) + δ_w2*(-3*t*w2 + u1*y + w1*x - w2*z)],
[                                                                                                              

In [476]:
#[D, U_i]
#l6
sp.expand(lie_alg_t(D, U1, der, dic_der) - lie_alg_t(U1, D, der, dic_der))


0

In [477]:
sp.expand(lie_alg_t(D, U2, der, dic_der) - lie_alg_t(U2, D, der, dic_der))

0

In [573]:
#[D, K]
l5 = []
v0 = D
for i in range(0,4):
    v1 = K[i]
    temp = (sp.expand(lie_alg_t(v0, v1, der, dic_der) - lie_alg_t(v1, v0, der, dic_der))).collect(der)
    l5.append(temp.collect(der))
sp.Matrix(l5)

Matrix([
[-2*t*x*δ_x - 2*t*y*δ_y - 2*t*z*δ_z + δ_t*(-t**2 - x**2 - y**2 - z**2) + δ_u1*(3*t*u1 - u1*z - u2*x - w2*y) + δ_u2*(3*t*u2 - u1*x + u2*z + w1*y) + δ_w1*(3*t*w1 + u2*y - w1*z - w2*x) + δ_w2*(3*t*w2 - u1*y - w1*x + w2*z)],
[  2*t*x*δ_t + 2*x*y*δ_y + 2*x*z*δ_z + δ_u1*(t*u2 - 3*u1*x + u2*z + w1*y) + δ_u2*(t*u1 - u1*z - 3*u2*x - w2*y) + δ_w1*(t*w2 - u1*y - 3*w1*x + w2*z) + δ_w2*(t*w1 + u2*y - w1*z - 3*w2*x) + δ_x*(t**2 + x**2 - y**2 - z**2)],
[2*t*y*δ_t + 2*x*y*δ_x + 2*y*z*δ_z + δ_u1*(t*w2 - 3*u1*y - w1*x + w2*z) + δ_u2*(-t*w1 - 3*u2*y + w1*z + w2*x) + δ_w1*(-t*u2 + u1*x - u2*z - 3*w1*y) + δ_w2*(t*u1 - u1*z - u2*x - 3*w2*y) + δ_y*(t**2 - x**2 + y**2 - z**2)],
[2*t*z*δ_t + 2*x*z*δ_x + 2*y*z*δ_y + δ_u1*(t*u1 - 3*u1*z - u2*x - w2*y) + δ_u2*(-t*u2 + u1*x - 3*u2*z - w1*y) + δ_w1*(t*w1 + u2*y - 3*w1*z - w2*x) + δ_w2*(-t*w2 + u1*y + w1*x - 3*w2*z) + δ_z*(t**2 - x**2 - y**2 + z**2)]])

In [479]:
#[D, M]
l4 = []
v0 = D
for i in range(0, 4):
    for j in range(0,4):
        v1 = M[i, j]
        temp = sp.expand(lie_alg_t(v0, v1, der, dic_der) - lie_alg_t(v1, v0, der, dic_der))
        l4.append(temp.collect(der))
sp.Matrix(l4).reshape(4, 4)
#0 basically

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [572]:
#[P, D] commutator
l1 = []
for i in range(0, 4):
    v0 = P[i]
    v1 = D
    temp = sp.expand(lie_alg_t(v0, v1, der, dic_der) - lie_alg_t(v1, v0, der, dic_der))
    l1.append(temp.collect(der))
sp.Matrix(l1)

Matrix([
[δ_t],
[δ_x],
[δ_y],
[δ_z]])

In [481]:
#[P, M]
l2 = []
for i in range(0,4):
    v0 = P[i]
    l2.append([])
    for j in range(0,4):
        for k in range(0, 4):
            v1 = M[j, k]
            temp = sp.expand(lie_alg_t(v0, v1, der, dic_der) - lie_alg_t(v1, v0, der, dic_der))
            l2[i].append(temp.collect(der))


In [482]:
sp.Matrix(l2[0]).reshape(4, 4)

Matrix([
[   0, δ_x, δ_y, δ_z],
[-δ_x,   0,   0,   0],
[-δ_y,   0,   0,   0],
[-δ_z,   0,   0,   0]])

In [483]:
sp.Matrix(l2[1]).reshape(4, 4)


Matrix([
[   0, δ_t,    0,    0],
[-δ_t,   0, -δ_y, -δ_z],
[   0, δ_y,    0,    0],
[   0, δ_z,    0,    0]])

In [484]:
sp.Matrix(l2[2]).reshape(4, 4)


Matrix([
[   0,    0, δ_t,    0],
[   0,    0, δ_x,    0],
[-δ_t, -δ_x,   0, -δ_z],
[   0,    0, δ_z,    0]])

In [485]:
sp.Matrix(l2[3]).reshape(4, 4)

Matrix([
[   0,    0,    0, δ_t],
[   0,    0,    0, δ_x],
[   0,    0,    0, δ_y],
[-δ_t, -δ_x, -δ_y,   0]])

In [486]:
D

t*δ_t - 3*u1*δ_u1/2 - 3*u2*δ_u2/2 - 3*w1*δ_w1/2 - 3*w2*δ_w2/2 + x*δ_x + y*δ_y + z*δ_z

In [579]:
#[K, P]
l3 = []
for i in range(0, 4):
    v0 = K[i]
    for j in range(0, 4):
        v1 = P[j]
        temp = sp.expand(lie_alg_t(v0, v1, der, dic_der)) - sp.expand(lie_alg_t(v1, v0, der, dic_der))
        l3.append(temp.collect(der))
sp.Matrix(l3).reshape(4, 4)

Matrix([
[2*t*δ_t - 3*u1*δ_u1 - 3*u2*δ_u2 - 3*w1*δ_w1 - 3*w2*δ_w2 + 2*x*δ_x + 2*y*δ_y + 2*z*δ_z,                              2*t*δ_x + u1*δ_u2 + u2*δ_u1 + w1*δ_w2 + w2*δ_w1 + 2*x*δ_t,                              2*t*δ_y + u1*δ_w2 - u2*δ_w1 - w1*δ_u2 + w2*δ_u1 + 2*y*δ_t,                              2*t*δ_z + u1*δ_u1 - u2*δ_u2 + w1*δ_w1 - w2*δ_w2 + 2*z*δ_t],
[                           -2*t*δ_x - u1*δ_u2 - u2*δ_u1 - w1*δ_w2 - w2*δ_w1 - 2*x*δ_t, -2*t*δ_t + 3*u1*δ_u1 + 3*u2*δ_u2 + 3*w1*δ_w1 + 3*w2*δ_w2 - 2*x*δ_x - 2*y*δ_y - 2*z*δ_z,                              u1*δ_w1 - u2*δ_w2 - w1*δ_u1 + w2*δ_u2 - 2*x*δ_y + 2*y*δ_x,                              u1*δ_u2 - u2*δ_u1 + w1*δ_w2 - w2*δ_w1 - 2*x*δ_z + 2*z*δ_x],
[                           -2*t*δ_y - u1*δ_w2 + u2*δ_w1 + w1*δ_u2 - w2*δ_u1 - 2*y*δ_t,                             -u1*δ_w1 + u2*δ_w2 + w1*δ_u1 - w2*δ_u2 + 2*x*δ_y - 2*y*δ_x, -2*t*δ_t + 3*u1*δ_u1 + 3*u2*δ_u2 + 3*w1*δ_w1 + 3*w2*δ_w2 - 2*x*δ_x - 2*y*δ_y - 2*z*δ_z,                      

In [580]:
M

Matrix([
[                                                             0,  t*δ_x + u1*δ_u2/2 + u2*δ_u1/2 + w1*δ_w2/2 + w2*δ_w1/2 + x*δ_t,  t*δ_y + u1*δ_w2/2 - u2*δ_w1/2 - w1*δ_u2/2 + w2*δ_u1/2 + y*δ_t, t*δ_z + u1*δ_u1/2 - u2*δ_u2/2 + w1*δ_w1/2 - w2*δ_w2/2 + z*δ_t],
[-t*δ_x - u1*δ_u2/2 - u2*δ_u1/2 - w1*δ_w2/2 - w2*δ_w1/2 - x*δ_t,                                                              0,  u1*δ_w1/2 - u2*δ_w2/2 - w1*δ_u1/2 + w2*δ_u2/2 - x*δ_y + y*δ_x, u1*δ_u2/2 - u2*δ_u1/2 + w1*δ_w2/2 - w2*δ_w1/2 - x*δ_z + z*δ_x],
[-t*δ_y - u1*δ_w2/2 + u2*δ_w1/2 + w1*δ_u2/2 - w2*δ_u1/2 - y*δ_t, -u1*δ_w1/2 + u2*δ_w2/2 + w1*δ_u1/2 - w2*δ_u2/2 + x*δ_y - y*δ_x,                                                              0, u1*δ_w2/2 + u2*δ_w1/2 - w1*δ_u2/2 - w2*δ_u1/2 - y*δ_z + z*δ_y],
[-t*δ_z - u1*δ_u1/2 + u2*δ_u2/2 - w1*δ_w1/2 + w2*δ_w2/2 - z*δ_t, -u1*δ_u2/2 + u2*δ_u1/2 - w1*δ_w2/2 + w2*δ_w1/2 + x*δ_z - z*δ_x, -u1*δ_w2/2 - u2*δ_w1/2 + w1*δ_u2/2 + w2*δ_u1/2 + y*δ_z - z*δ_y,                            

In [488]:
l_t = []
for i in range(0, 4):
    v0 = K[i]
    for j in range(0,4):
        v1 = K[j]
        temp = sp.expand(lie_alg_t(v0, v1, der, dic_der) - lie_alg_t(v1, v0, der, dic_der))
        l_t.append(temp.collect(der))
sp.Matrix(l_t).reshape(4, 4)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])